## Lid Driven and free slip boundary conditions in annulus geometry

A example of the annulus convection model using a wrapper

In [ ]:
import underworld as uw
from underworld import function as fn
import glucifer
import math
import numpy as np

In [ ]:
from unsupported import annulus

In [ ]:
model = annulus.AnnulusConvection(elRes=(30, 300), outputPath='output')

In [ ]:
model.meshSets.keys()

In [ ]:
model.initial_temperature_distribution(k=4)
# load from hdf5 file
# model.initial_temperature_distribution(h5file='annulus_output/temperature-100.h5')
# model.fields['tDot'].load('annulus_output/tDot-100.h5', interpolate=True)
# tField = model.fields['temperature']
# tDot = model.fields['tDot']
# tField.data[model.meshSets['outerNodes'].data] = 0.0
# tField.data[model.meshSets['innerNodes'].data] = 1.0
# tDot.data[model.meshSets['boundaryNodes'].data] = 0.0

In [ ]:
model.view.show()

In [ ]:
model.parameter_setup(Ra=1e4, Di=0.25, viscosity_mode=0, eqn='EBA')
model.model_init()

In [ ]:
model.simulate(maxtimesteps=10, visualiseEvery=5, checkpointEvery=10 )

In [ ]:
model.view.show()

In [ ]:
fig= glucifer.Figure()
fig.append(glucifer.objects.Surface(mesh=model.mesh, fn=model.adiabatic_heating, onMesh=True ))
fig.show()

In [ ]:
vis = glucifer.lavavu.Viewer(database=model.animation.filename)

In [ ]:
vis.control.Panel()
vis.control.ObjectList()
vis.control.TimeStepper()
vis.control.show()